In [1]:
import pandas as pd

In [2]:
stats=pd.read_csv(r"datasets/player_mvp_stats_adv.csv")
del stats['Unnamed: 0']
stats22_24=pd.read_csv(r"stats_2024.csv")
del stats22_24['Unnamed: 0']

In [3]:
stats=pd.concat([stats,stats22_24],ignore_index=True)

In [4]:
pd.set_option('display.max_rows', None)
pd.isnull(stats).sum()

Player     0
Pos        0
Age        0
Tm         0
G          0
GS         0
MP         0
FG         0
FGA        0
FG%        0
3P         0
3PA        0
3P%        0
2P         0
2PA        0
2P%        0
eFG%       0
FT         0
FTA        0
FT%        0
ORB        0
DRB        0
TRB        0
AST        0
STL        0
BLK        0
TOV        0
PF         0
PTS        0
Year       0
Pts Won    0
Pts Max    0
Share      0
Team       0
W          0
L          0
W/L%       0
GB         0
PS/G       0
PA/G       0
SRS        0
T_MP       0
PER        0
TS%        0
3PAr       0
FTr        0
ORB%       0
DRB%       0
TRB%       0
AST%       0
STL%       0
BLK%       0
TOV%       0
USG%       0
OWS        0
DWS        0
WS         0
WS/48      0
OBPM       0
DBPM       0
BPM        0
VORP       0
dtype: int64

In [5]:
stats.columns

Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year',
       'Pts Won', 'Pts Max', 'Share', 'Team', 'W', 'L', 'W/L%', 'GB', 'PS/G',
       'PA/G', 'SRS', 'T_MP', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%',
       'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS',
       'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP'],
      dtype='object')

In [6]:
predictors=['Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
        '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
        'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year',
        'W', 'L', 'W/L%', 'GB', 'PS/G',
        'PA/G', 'SRS', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%',
        'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS',
        'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP']

In [7]:
train=stats[stats['Year']<2024]
test=stats[stats['Year']==2024]

In [8]:
from sklearn.linear_model import Ridge

reg = Ridge(alpha=.1)

In [9]:
reg.fit(train[predictors], train["Share"])

Ridge(alpha=0.1)

In [10]:
preds=reg.predict(test[predictors])

In [11]:
preds=pd.DataFrame(preds,columns=['predictions'], index=test.index)

In [12]:
pd.set_option('display.max_rows', 15)
preds

,predictions
14131,0.010700
14132,0.003517
14133,-0.000277
14134,-0.016613
14135,-0.025930
...,...
15803,-0.009421
15804,-0.003335
15805,-0.008603
15806,0.007627


In [13]:
combo=pd.concat([test[["Player","Share"]], preds], axis=1)

In [14]:
combo

,Player,Share,predictions
14131,AJ Griffin,0.0,0.010700
14132,Bogdan Bogdanović,0.0,0.003517
14133,Bruno Fernando,0.0,-0.000277
14134,Clint Capela,0.0,-0.016613
14135,De'Andre Hunter,0.0,-0.025930
...,...,...,...
15803,Marvin Bagley III,0.0,-0.009421
15804,Patrick Baldwin Jr,0.0,-0.003335
15805,Richaun Holmes,0.0,-0.008603
15806,Tristan Vukcevic,0.0,0.007627


In [15]:
combo.sort_values("Share", ascending=False).head(10)

,Player,Share,predictions
14541,Nikola Jokić,0.935,0.255667
15274,Shai Gilgeous-Alexander,0.646,0.155579
14485,Luka Dončić,0.572,0.205676
15045,Giannis Antetokounmpo,0.194,0.230775
15204,Jalen Brunson,0.143,0.119055
14195,Jayson Tatum,0.087,0.102609
15091,Anthony Edwards,0.018,0.073431
15553,Domantas Sabonis,0.003,0.143784
15438,Kevin Durant,0.001,0.091578
15263,Isaiah Joe,0.000,0.004789


In [16]:
from sklearn.metrics import mean_squared_error

mean_squared_error(combo["Share"],combo['predictions'])

0.0020033793961132863

In [17]:
combo['Share'].value_counts()

Share
0.000    563
0.087      1
0.572      1
0.935      1
0.194      1
0.018      1
0.143      1
0.646      1
0.001      1
0.003      1
Name: count, dtype: int64

In [18]:
combo=combo.sort_values('Share',ascending=False)
combo['Rk']=list(range(1,combo.shape[0]+1))

In [19]:
combo.head(10)

,Player,Share,predictions,Rk
14541,Nikola Jokić,0.935,0.255667,1
15274,Shai Gilgeous-Alexander,0.646,0.155579,2
14485,Luka Dončić,0.572,0.205676,3
15045,Giannis Antetokounmpo,0.194,0.230775,4
15204,Jalen Brunson,0.143,0.119055,5
14195,Jayson Tatum,0.087,0.102609,6
15091,Anthony Edwards,0.018,0.073431,7
15553,Domantas Sabonis,0.003,0.143784,8
15438,Kevin Durant,0.001,0.091578,9
15263,Isaiah Joe,0.000,0.004789,10


In [20]:
combo=combo.sort_values('predictions', ascending=False)
combo['Predicted_Rk']=list(range(1,combo.shape[0]+1))

In [21]:
combo.head(10)

,Player,Share,predictions,Rk,Predicted_Rk
14541,Nikola Jokić,0.935,0.255667,1,1
15045,Giannis Antetokounmpo,0.194,0.230775,4,2
14485,Luka Dončić,0.572,0.205676,3,3
15274,Shai Gilgeous-Alexander,0.646,0.155579,2,4
15553,Domantas Sabonis,0.003,0.143784,8,5
14870,LeBron James,0.000,0.121944,477,6
15204,Jalen Brunson,0.143,0.119055,5,7
14858,Anthony Davis,0.000,0.108436,439,8
15376,Joel Embiid,0.000,0.102751,45,9
14195,Jayson Tatum,0.087,0.102609,6,10


In [22]:
combo.sort_values("Share",ascending=False).head(10)

,Player,Share,predictions,Rk,Predicted_Rk
14541,Nikola Jokić,0.935,0.255667,1,1
15274,Shai Gilgeous-Alexander,0.646,0.155579,2,4
14485,Luka Dončić,0.572,0.205676,3,3
15045,Giannis Antetokounmpo,0.194,0.230775,4,2
15204,Jalen Brunson,0.143,0.119055,5,7
14195,Jayson Tatum,0.087,0.102609,6,10
15091,Anthony Edwards,0.018,0.073431,7,16
15553,Domantas Sabonis,0.003,0.143784,8,5
15438,Kevin Durant,0.001,0.091578,9,11
14751,Ben Sheppard,0.000,-0.007204,547,387


In [23]:
def find_ap(combo:pd.DataFrame):
    """find_ap

    Args:
        combo (pd.DataFrame): 

    Returns:
        float: 
    """
    actual=combo.sort_values("Share", ascending=False).head(5)
    predicted=combo.sort_values("predictions", ascending=False)
    ps=[]
    found=0
    seen=1
    for i,r in predicted.iterrows():
        if r['Player'] in actual['Player'].values:
            found+=1
            ps.append(found/seen)
        seen+=1
        
    return sum(ps)/len(ps)

In [24]:
find_ap(combo=combo)

0.9428571428571428

In [25]:
predictors

['Age',
 'G',
 'GS',
 'MP',
 'FG',
 'FGA',
 'FG%',
 '3P',
 '3PA',
 '3P%',
 '2P',
 '2PA',
 '2P%',
 'eFG%',
 'FT',
 'FTA',
 'FT%',
 'ORB',
 'DRB',
 'TRB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PTS',
 'Year',
 'W',
 'L',
 'W/L%',
 'GB',
 'PS/G',
 'PA/G',
 'SRS',
 'PER',
 'TS%',
 '3PAr',
 'FTr',
 'ORB%',
 'DRB%',
 'TRB%',
 'AST%',
 'STL%',
 'BLK%',
 'TOV%',
 'USG%',
 'OWS',
 'DWS',
 'WS',
 'WS/48',
 'OBPM',
 'DBPM',
 'BPM',
 'VORP']

In [26]:
years=list(range(1991,2025))

In [27]:
aps=[]
all_preds=[]
for year in years[5:]:
    train=stats[stats["Year"]<year]
    test=stats[stats["Year"]==year]
    reg.fit(train[predictors],train["Share"])
    preds=reg.predict(test[predictors])
    preds=pd.DataFrame(preds,columns=['predictions'],index=test.index)
    combo=pd.concat([test[["Player","Share"]],preds],axis=1)
    all_preds.append(combo)
    aps.append(find_ap(combo))


In [28]:
sum(aps)/len(aps)

0.7842064667303726

In [29]:
def add_ranks(combo:pd.DataFrame)->pd.DataFrame:
    combo=combo.sort_values('Share',ascending=False)
    combo['Rk']=list(range(1,combo.shape[0]+1))
    combo=combo.sort_values('predictions', ascending=False)
    combo['Predicted_Rk']=list(range(1,combo.shape[0]+1))
    combo['Diff']=combo["Rk"]-combo['Predicted_Rk']
    return combo

In [30]:
ranking=add_ranks(all_preds[1])
ranking[ranking['Rk']<6].sort_values('Diff', ascending=False)

,Player,Share,predictions,Rk,Predicted_Rk,Diff
1670,Michael Jordan,0.832,0.254972,2,1,1
3610,Grant Hill,0.327,0.218286,3,3,0
13135,Karl Malone,0.857,0.247365,1,2,-1
6833,Tim Hardaway,0.207,0.122020,4,8,-4
1377,Glen Rice,0.117,0.025898,5,64,-59


In [31]:
def backtest(stats, model, years, predictors):
    aps=[]
    all_preds=[]
    for year in years[5:]:
        train=stats[stats["Year"]<year]
        test=stats[stats["Year"]==year]
        model.fit(train[predictors],train["Share"])
        preds=reg.predict(test[predictors])
        preds=pd.DataFrame(preds,columns=['predictions'],index=test.index)
        combo=pd.concat([test[["Player","Share"]],preds],axis=1)
        combo=add_ranks(combo)
        all_preds.append(combo)
        aps.append(find_ap(combo))
    return sum(aps)/len(aps),aps,pd.concat(all_preds)

In [32]:
mean_ap,aps,all_preds=backtest(stats,reg, years[5:], predictors)

In [33]:
mean_ap

0.8012150815825096

In [34]:
all_preds[all_preds['Rk']<=5].sort_values("Diff").head(10)

,Player,Share,predictions,Rk,Predicted_Rk,Diff
10697,Steve Nash,0.839,0.033186,1,53,-52
11606,Peja Stojaković,0.228,0.031658,4,52,-48
10715,Steve Nash,0.739,0.051923,1,38,-37
8345,Jason Kidd,0.712,0.048377,2,33,-31
10730,Steve Nash,0.785,0.074515,2,21,-19
3744,Chauncey Billups,0.344,0.079492,5,22,-17
4439,Stephen Curry,0.173,0.073753,5,21,-16
12212,Tony Parker,0.274,0.059673,5,21,-16
10935,Chris Paul,0.138,0.056352,5,20,-15
15396,Devin Booker,0.216,0.073048,4,16,-12


In [35]:
#Algorithm is undervalueing point guards

In [36]:
# from sklearn.preprocessing import StandardScaler
# stats = stats[stats['FG%'] != 0]
# scaler = StandardScaler()
# scaled_features = scaler.fit_transform(stats[predictors])  # Exclude non-numeric columns
# stats_scaled = pd.DataFrame(scaled_features, columns=predictors)
# stats_scaled['Share']=stats['Share']

In [37]:
pd.set_option('display.max_rows', None)
pd.concat([pd.Series(reg.coef_), pd.Series(predictors)], axis=1).sort_values(0, ascending=False)

,0,1
13,0.159391,eFG%
29,0.071013,W/L%
18,0.029933,DRB
53,0.025721,VORP
15,0.023582,FTA
16,0.020297,FT%
49,0.017533,WS/48
5,0.014872,FGA
23,0.013517,TOV
17,0.010830,ORB


In [38]:
predictors

['Age',
 'G',
 'GS',
 'MP',
 'FG',
 'FGA',
 'FG%',
 '3P',
 '3PA',
 '3P%',
 '2P',
 '2PA',
 '2P%',
 'eFG%',
 'FT',
 'FTA',
 'FT%',
 'ORB',
 'DRB',
 'TRB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PTS',
 'Year',
 'W',
 'L',
 'W/L%',
 'GB',
 'PS/G',
 'PA/G',
 'SRS',
 'PER',
 'TS%',
 '3PAr',
 'FTr',
 'ORB%',
 'DRB%',
 'TRB%',
 'AST%',
 'STL%',
 'BLK%',
 'TOV%',
 'USG%',
 'OWS',
 'DWS',
 'WS',
 'WS/48',
 'OBPM',
 'DBPM',
 'BPM',
 'VORP']

In [39]:
stat_ratios=stats[["PTS","AST","TRB", "STL","BLK","3P","WS","WS/48", "eFG%", "VORP","BPM",'PER',"Year"]].groupby("Year").apply(lambda x: x/x.mean())

C:\Users\zac15\AppData\Local\Temp\ipykernel_15552\1253354308.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stat_ratios=stats[["PTS","AST","TRB", "STL","BLK","3P","WS","WS/48", "eFG%", "VORP","BPM",'PER',"Year"]].groupby("Year").apply(lambda x: x/x.mean())


In [40]:
stat_ratios.index=stat_ratios.index.droplevel()

In [41]:
pd.set_option('display.max_rows', 20)
stat_ratios

,PTS,AST,TRB,STL,BLK,3P,WS,WS/48,eFG%,VORP,BPM,PER,Year
0,1.692601,2.010078,0.866690,2.608773,1.346939,5.594452,2.166399,1.468403,1.024506,3.728119,-1.296067,1.220476,1.0
1,2.884104,1.542618,2.437566,2.059558,1.795918,5.085865,3.920150,2.240580,1.060943,7.743017,-3.342488,1.770444,1.0
2,0.679268,0.327222,0.622934,0.549215,0.673469,0.000000,0.653358,1.000033,1.054513,-0.143389,1.637137,0.934192,1.0
3,0.055678,0.000000,0.135420,0.000000,0.673469,0.000000,0.000000,1.443085,0.000000,0.000000,3.547130,0.655441,1.0
4,1.514433,1.262142,0.541681,0.823823,0.224490,1.017173,1.375491,1.291182,1.005216,0.716946,0.613926,1.235544,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15803,1.389012,0.549616,1.838084,0.676523,1.737847,0.212560,1.222609,1.612161,1.155861,0.762921,0.229236,1.508906,1.0
15804,0.522364,0.399720,0.948689,0.845653,0.993056,0.850242,0.090564,0.251074,0.931271,-0.572191,2.636209,0.754453,1.0
15805,0.593595,0.299790,1.363740,0.507392,1.241319,0.000000,0.588664,1.506446,1.086161,-0.381460,1.891193,1.127791,1.0
15806,1.009111,0.649546,1.067275,0.845653,1.737847,1.062802,0.045282,0.594650,0.981610,-0.190730,2.063120,1.081124,1.0


In [42]:
stats.columns

Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year',
       'Pts Won', 'Pts Max', 'Share', 'Team', 'W', 'L', 'W/L%', 'GB', 'PS/G',
       'PA/G', 'SRS', 'T_MP', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%',
       'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS',
       'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP'],
      dtype='object')

In [43]:
stats_copy=stats.copy()
stats_copy[
    ['PTS_R',
 'AST_R',
 'TRB_R',
 'STL_R',
 'BLK_R',
 '3P_R',
 'WS_R',
 'WS/48_R',
 'eFG%_R',
 'VORP_R',
 'BPM_R',
 'PER_R']
    ]=stat_ratios[
    ["PTS","AST","TRB", "STL","BLK","3P","WS","WS/48", "eFG%", "VORP","BPM",'PER']]

In [44]:
stats_copy.columns

Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year',
       'Pts Won', 'Pts Max', 'Share', 'Team', 'W', 'L', 'W/L%', 'GB', 'PS/G',
       'PA/G', 'SRS', 'T_MP', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%',
       'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS',
       'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP', 'PTS_R', 'AST_R', 'TRB_R',
       'STL_R', 'BLK_R', '3P_R', 'WS_R', 'WS/48_R', 'eFG%_R', 'VORP_R',
       'BPM_R', 'PER_R'],
      dtype='object')

In [45]:
stats_copy= stats_copy[
    (stats['PTS'] > 0) &  # Players who scored at least some points
    (stats['G'] >= 10) &  # Players who played in at least 10 games
    (stats['MP'] >= 20)  # Players with meaningful playing time
    
]

In [46]:
#stats[["PTS_R","AST_R","STL_R","BLK_R","3P_R","VORP_R"]]=stat_ratios[["PTS","AST","STL","BLK","3P","VORP"]]

In [47]:
stats_copy.columns

Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year',
       'Pts Won', 'Pts Max', 'Share', 'Team', 'W', 'L', 'W/L%', 'GB', 'PS/G',
       'PA/G', 'SRS', 'T_MP', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%',
       'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS',
       'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP', 'PTS_R', 'AST_R', 'TRB_R',
       'STL_R', 'BLK_R', '3P_R', 'WS_R', 'WS/48_R', 'eFG%_R', 'VORP_R',
       'BPM_R', 'PER_R'],
      dtype='object')

In [48]:
test_predictors=predictors+['PTS_R',
 'AST_R',
 'TRB_R',
 'STL_R',
 'BLK_R',
 '3P_R',
 'WS_R',
 'WS/48_R',
 'eFG%_R',
 'VORP_R',
 'BPM_R',
 'PER_R']

In [49]:
stats.columns

Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year',
       'Pts Won', 'Pts Max', 'Share', 'Team', 'W', 'L', 'W/L%', 'GB', 'PS/G',
       'PA/G', 'SRS', 'T_MP', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%',
       'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS',
       'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP'],
      dtype='object')

In [50]:
mean_ap,aps,all_preds=backtest(stats_copy,reg, years[5:], test_predictors)

In [51]:
mean_ap

0.8217302526877335

In [52]:
all_preds[all_preds['Rk']<=5].sort_values("Diff").head(10)

,Player,Share,predictions,Rk,Predicted_Rk,Diff
10697,Steve Nash,0.839,0.039903,1,37,-36
8345,Jason Kidd,0.712,0.026865,2,36,-34
10715,Steve Nash,0.739,0.082627,1,21,-20
10935,Chris Paul,0.138,0.072570,5,23,-18
1923,Joakim Noah,0.258,0.102380,4,18,-14
12212,Tony Parker,0.274,0.101759,5,18,-13
11606,Peja Stojaković,0.228,0.074585,4,17,-13
3744,Chauncey Billups,0.344,0.104754,5,15,-10
6241,Kobe Bryant,0.291,0.116830,4,14,-10
15396,Devin Booker,0.216,0.107310,4,13,-9


In [53]:
f_stats=stats_copy.copy()#filtered stats

In [54]:
p_test=['Age',
 'G',
 'GS',
 'MP',
 'FG',
 'FGA',
 'FG%',
 '3P',
 '3PA',
 '3P%',
 '2P',
 '2PA',
 '2P%',
 'eFG%',
 'FT',
 'FTA',
 'FT%',
 'ORB',
 'DRB',
 'TRB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PTS',
 'W',
 'L',
 'W/L%',
 'GB',
 'PS/G',
 'PA/G',
 'SRS',
 'PER',
 'TS%',
 '3PAr',
 'FTr',
 'ORB%',
 'DRB%',
 'TRB%',
 'AST%',
 'STL%',
 'BLK%',
 'TOV%',
 'USG%',
 'OWS',
 'DWS',
 'WS',
 'WS/48',
 'OBPM',
 'DBPM',
 'BPM',
 'VORP',
 'PTS_R',
 'AST_R',
 'TRB_R',
 'STL_R',
 'BLK_R',
 '3P_R',
 'WS_R',
 'WS/48_R',
 'eFG%_R',
 'VORP_R',
 'BPM_R',
 'PER_R']

In [55]:
from sklearn.preprocessing import StandardScaler
normalized_stats=f_stats.copy()
scaler=StandardScaler()
normalized_stats[p_test]=scaler.fit_transform(normalized_stats[p_test])

In [56]:
test_predictors

['Age',
 'G',
 'GS',
 'MP',
 'FG',
 'FGA',
 'FG%',
 '3P',
 '3PA',
 '3P%',
 '2P',
 '2PA',
 '2P%',
 'eFG%',
 'FT',
 'FTA',
 'FT%',
 'ORB',
 'DRB',
 'TRB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PTS',
 'Year',
 'W',
 'L',
 'W/L%',
 'GB',
 'PS/G',
 'PA/G',
 'SRS',
 'PER',
 'TS%',
 '3PAr',
 'FTr',
 'ORB%',
 'DRB%',
 'TRB%',
 'AST%',
 'STL%',
 'BLK%',
 'TOV%',
 'USG%',
 'OWS',
 'DWS',
 'WS',
 'WS/48',
 'OBPM',
 'DBPM',
 'BPM',
 'VORP',
 'PTS_R',
 'AST_R',
 'TRB_R',
 'STL_R',
 'BLK_R',
 '3P_R',
 'WS_R',
 'WS/48_R',
 'eFG%_R',
 'VORP_R',
 'BPM_R',
 'PER_R']

In [57]:
test_predictors
mean_ap,aps,all_preds=backtest(normalized_stats,reg, years[5:], [t for t in test_predictors if t !='Year'])

In [58]:
mean_ap

0.8349180533755343

In [59]:
#mean top 5 accuracy is 83.5%

In [60]:
all_preds[all_preds['Rk']<=5].sort_values("Diff").head(10)

,Player,Share,predictions,Rk,Predicted_Rk,Diff
10697,Steve Nash,0.839,0.035409,1,40,-39
8345,Jason Kidd,0.712,0.020904,2,39,-37
10715,Steve Nash,0.739,0.083617,1,21,-20
10935,Chris Paul,0.138,0.072958,5,23,-18
11606,Peja Stojaković,0.228,0.071918,4,20,-16
1923,Joakim Noah,0.258,0.099521,4,18,-14
12212,Tony Parker,0.274,0.100636,5,17,-12
3744,Chauncey Billups,0.344,0.103930,5,15,-10
15396,Devin Booker,0.216,0.103113,4,13,-9
9152,Carmelo Anthony,0.393,0.103860,3,11,-8


In [61]:
pd.set_option('display.max_rows',None)
# jason=stats.loc[stats['Share']==0.135]
# stats[stats['Year']==1999].sort_values("Share",ascending=False).head(10)
stats.loc[(stats['Player']=='Glen Rice') & (stats['Share']==0.117)]
stats[stats['Year']==2005].sort_values('Share', ascending=False)[['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%','TRB', 'AST', 'STL', 'BLK', 'TOV', 'PTS', 'Year',
       'Pts Won', 'Pts Max', 'Share', 'W', 'L','GB', 'W/L%', 'PER', 'WS']].head(10)

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,Year,Pts Won,Pts Max,Share,W,L,GB,W/L%,PER,WS
10697,Steve Nash,PG,30,PHO,75,75,34.3,5.7,11.4,0.502,...,2005,1066.0,1270.0,0.839,62,20,0.0,0.756,22.0,10.9
6960,Shaquille O'Neal,C,32,MIA,73,73,34.1,9.0,15.0,0.601,...,2005,1032.0,1270.0,0.813,59,23,0.0,0.720,27.0,11.0
2769,Dirk Nowitzki,PF,26,DAL,78,78,38.7,8.5,18.5,0.459,...,2005,349.0,1270.0,0.275,58,24,1.0,0.707,26.1,15.6
12095,Tim Duncan,PF,28,SAS,66,66,33.4,7.8,15.8,0.496,...,2005,328.0,1270.0,0.258,59,23,0.0,0.720,27.0,11.2
10209,Allen Iverson,PG,29,PHI,75,75,42.3,10.3,24.2,0.424,...,2005,240.0,1270.0,0.189,43,39,2.0,0.524,23.2,9.0
2270,LeBron James,SF,20,CLE,80,80,42.4,9.9,21.1,0.472,...,2005,93.0,1270.0,0.073,42,40,12.0,0.512,25.7,14.3
4695,Tracy McGrady,SF,25,HOU,78,78,40.8,9.2,21.3,0.431,...,2005,44.0,1270.0,0.035,51,31,8.0,0.622,22.9,12.0
6953,Dwyane Wade,SG,23,MIA,77,77,38.6,8.2,17.1,0.478,...,2005,43.0,1270.0,0.034,59,23,0.0,0.720,23.1,11.0
12565,Ray Allen,SG,29,SEA,78,78,39.3,8.2,19.2,0.428,...,2005,41.0,1270.0,0.032,52,30,0.0,0.634,20.9,10.7
10687,Amar'e Stoudemire,C,22,PHO,80,80,36.1,9.3,16.7,0.559,...,2005,41.0,1270.0,0.032,62,20,0.0,0.756,26.6,14.6


In [62]:
mean_ap,aps,all_preds=backtest(stats_copy,reg, years[5:], test_predictors)

In [63]:
mean_ap

0.8217302526877335

In [64]:
mean_ap,aps,all_preds=backtest(stats_copy,reg, years[5:], test_predictors)

In [65]:
test=f_stats[f_stats["Year"]==2024]
filtered_test=test[
    (test['PTS'] > 15) &  # Players who scored at least some points
    (test['G'] >= 65) &  # Players who played in at least 10 games
    (test['MP'] >= 20) # Players with meaningful playing time
]
filtered_test.head(3)

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,TRB_R,STL_R,BLK_R,3P_R,WS_R,WS/48_R,eFG%_R,VORP_R,BPM_R,PER_R
14132,Bogdan Bogdanović,SG,31,ATL,79,33,30.4,6.0,13.9,0.428,...,1.007982,2.029568,0.744792,3.188406,1.765991,1.043940,1.039694,2.670223,-0.114618,1.143347
14136,Dejounte Murray,SG,27,ATL,78,78,35.7,8.6,18.8,0.459,...,1.571266,2.367830,0.744792,2.763285,2.218809,1.110012,1.022269,4.958986,-0.974251,1.376683
14190,Derrick White,SG,29,BOS,73,73,32.6,5.3,11.5,0.461,...,1.245154,1.691307,2.979167,2.869565,3.848955,2.259668,1.119074,6.675559,-2.177738,1.314460


In [66]:
filtered_test=test.copy()
scaler=StandardScaler()
filtered_test[p_test]=scaler.fit_transform(filtered_test[p_test])

In [67]:
year=2024
train=normalized_stats[normalized_stats["Year"]<year]
# test=normalized_stats[normalized_stats["Year"]==year]
# filtered_test=test[
#     (test['PTS'] > 15) &  # Players who scored at least some points
#     (test['G'] >= 65) &  # Players who played in at least 10 games
#     (test['MP'] >= 20) # Players with meaningful playing time
# ]
reg.fit(train[test_predictors],train["Share"])
preds=reg.predict(filtered_test[test_predictors])


In [68]:
preds=pd.DataFrame(preds,columns=['predictions'], index=test.index)
combo=pd.concat([test[["Player","Share"]], preds], axis=1)
combo.sort_values("Share", ascending=False).head(10)

,Player,Share,predictions
14541,Nikola Jokić,0.935,0.315865
15274,Shai Gilgeous-Alexander,0.646,0.205528
14485,Luka Dončić,0.572,0.248458
15045,Giannis Antetokounmpo,0.194,0.300380
15204,Jalen Brunson,0.143,0.138617
14195,Jayson Tatum,0.087,0.109690
15091,Anthony Edwards,0.018,0.058639
15553,Domantas Sabonis,0.003,0.145347
15438,Kevin Durant,0.001,0.108315
15206,Josh Hart,0.000,-0.014118


In [69]:
all_preds[all_preds['Rk']<=5].sort_values("Diff").head(10)
# pd.set_option('display.max_rows', None)
df=pd.concat([pd.Series(reg.coef_), pd.Series(predictors)], axis=1).sort_values(0, ascending=False)
df.to_clipboard()

In [70]:
stats["NPos"]=stats["Pos"].astype("category").cat.codes
stats["NTm"]=stats["Tm"].astype("category").cat.codes

In [71]:
#Random Forest
from sklearn.ensemble import RandomForestRegressor

rf=RandomForestRegressor(n_estimators=500, random_state=15, min_samples_split=7,n_jobs=-1)

mean_ap,aps,all_preds=backtest(stats,rf, years[5:], predictors)

ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- 3P_R
- AST_R
- BLK_R
- BPM_R
- PER_R
- ...
